In [64]:
import pyupbit
import time
import json
import requests
import pandas as pd
import mysql.connector

##### DB연결

In [67]:
# MySQL 서버에 연결
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="deepCoin"
)

# 연결이 제대로 되었는지 확인
if mydb.is_connected():
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
else:
    print("MySQL 데이터베이스 연결에 실패했습니다.")

# 커서 생성
mycursor = mydb.cursor()


MySQL 데이터베이스에 성공적으로 연결되었습니다.


##### DB 종목마스터 가져오기

In [50]:
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

##### 종목별 일봉데이터 가져오기

In [83]:
for market in market_list:
    pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=None,
                   to=None, period=0.5)

In [62]:
market = "KRW-ASTR"
df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=None,
                   to=None, period=0.5)
df['date'] = df.index
df['market'] = market
df = df[['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']]

values = df.values.tolist()

In [69]:
mydb.close()

In [68]:
# KRW 종목 리스트 가져오기
columns = ['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']

# 삽입할 테이블 이름
table_name = "c_krw_day"

# 데이터 삽입 쿼리 생성
sql = "INSERT INTO {} ({}) VALUES ({})".format(
    table_name,
    ', '.join(columns),
    ', '.join(['%s'] * len(columns))
)

# 데이터 삽입
mycursor.executemany(sql, values)

# 변경 사항 커밋
mydb.commit()
print(mycursor.rowcount, "레코드가 삽입되었습니다.")
mydb.close()

ProgrammingError: 1054 (42S22): Unknown column 'high' in 'field list'

##### 종목마스터 수집

In [4]:
pyupbit.get_tickers(fiat="KRW")

119

In [28]:
url = "https://api.upbit.com/v1/market/all?isDetails=true"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)
df = pd.DataFrame(mst_json)
df = df[['market', 'korean_name', 'english_name', 'market_warning']]
df = df[df['market'].str.contains('KRW')].reset_index(drop=True)


# DB생성용 Master 항목당 최대 길이 출력
max_length = df['market'].str.len().max()
max_string = df[df['market'].str.len() == max_length]['market'].iloc[0]
# print(max_string, len(max_string))

# DataFrame을 튜플의 리스트로 변환
values = df.values.tolist()

# 컬럼명 리스트
columns = df.columns.tolist()

# 삽입할 테이블 이름
table_name = "c_krw_mst"

In [29]:
values

[['KRW-BTC', '비트코인', 'Bitcoin', 'NONE'],
 ['KRW-ETH', '이더리움', 'Ethereum', 'NONE'],
 ['KRW-NEO', '네오', 'NEO', 'NONE'],
 ['KRW-MTL', '메탈', 'Metal', 'NONE'],
 ['KRW-XRP', '리플', 'Ripple', 'NONE'],
 ['KRW-ETC', '이더리움클래식', 'Ethereum Classic', 'NONE'],
 ['KRW-SNT', '스테이터스네트워크토큰', 'Status Network Token', 'NONE'],
 ['KRW-WAVES', '웨이브', 'Waves', 'NONE'],
 ['KRW-XEM', '넴', 'NEM', 'NONE'],
 ['KRW-QTUM', '퀀텀', 'Qtum', 'NONE'],
 ['KRW-LSK', '리스크', 'Lisk', 'NONE'],
 ['KRW-STEEM', '스팀', 'Steem', 'NONE'],
 ['KRW-XLM', '스텔라루멘', 'Lumen', 'NONE'],
 ['KRW-ARDR', '아더', 'Ardor', 'NONE'],
 ['KRW-ARK', '아크', 'Ark', 'NONE'],
 ['KRW-STORJ', '스토리지', 'Storj', 'NONE'],
 ['KRW-GRS', '그로스톨코인', 'Groestlcoin', 'NONE'],
 ['KRW-ADA', '에이다', 'Ada', 'NONE'],
 ['KRW-SBD', '스팀달러', 'SteemDollars', 'NONE'],
 ['KRW-POWR', '파워렛저', 'Power ledger', 'NONE'],
 ['KRW-BTG', '비트코인골드', 'Bitcoin Gold', 'NONE'],
 ['KRW-ICX', '아이콘', 'Icon', 'NONE'],
 ['KRW-EOS', '이오스', 'EOS', 'NONE'],
 ['KRW-TRX', '트론', 'TRON', 'NONE'],
 ['KRW-SC', '시아코인',

In [5]:
# KRW 종목 리스트 가져오기
columns = pyupbit.get_tickers(fiat="KRW")

# 삽입할 테이블 이름
table_name = "c_krw_mst"

# 데이터 삽입 쿼리 생성
sql = "INSERT INTO {} ({}) VALUES ({})".format(
    table_name,
    ', '.join(columns),
    ', '.join(['%s'] * len(columns))
)

# 데이터 삽입
mycursor.executemany(sql, values)

# 변경 사항 커밋
mydb.commit()
print(mycursor.rowcount, "레코드가 삽입되었습니다.")
mydb.close()

NameError: name 'table_name' is not defined

In [ ]:
market_list

['KRW-1INCH',
 'KRW-AAVE',
 'KRW-ADA',
 'KRW-AERGO',
 'KRW-AHT',
 'KRW-ALGO',
 'KRW-ANKR',
 'KRW-APT',
 'KRW-AQT',
 'KRW-ARB',
 'KRW-ARDR',
 'KRW-ARK',
 'KRW-ASTR',
 'KRW-ATOM',
 'KRW-AVAX',
 'KRW-AXS',
 'KRW-BAT',
 'KRW-BCH',
 'KRW-BLUR',
 'KRW-BORA',
 'KRW-BSV',
 'KRW-BTC',
 'KRW-BTG',
 'KRW-BTT',
 'KRW-CBK',
 'KRW-CELO',
 'KRW-CHZ',
 'KRW-CRE',
 'KRW-CRO',
 'KRW-CTC',
 'KRW-CVC',
 'KRW-DKA',
 'KRW-DOGE',
 'KRW-DOT',
 'KRW-EGLD',
 'KRW-ELF',
 'KRW-EOS',
 'KRW-ETC',
 'KRW-ETH',
 'KRW-FCT2',
 'KRW-FLOW',
 'KRW-GAS',
 'KRW-GLM',
 'KRW-GMT',
 'KRW-GRS',
 'KRW-GRT',
 'KRW-HBAR',
 'KRW-HIFI',
 'KRW-HIVE',
 'KRW-HPO',
 'KRW-HUNT',
 'KRW-ICX',
 'KRW-IMX',
 'KRW-IOST',
 'KRW-IOTA',
 'KRW-IQ',
 'KRW-JST',
 'KRW-KAVA',
 'KRW-KNC',
 'KRW-LINK',
 'KRW-LOOM',
 'KRW-LSK',
 'KRW-MANA',
 'KRW-MASK',
 'KRW-MATIC',
 'KRW-MBL',
 'KRW-MED',
 'KRW-META',
 'KRW-MINA',
 'KRW-MLK',
 'KRW-MOC',
 'KRW-MTL',
 'KRW-MVL',
 'KRW-NEAR',
 'KRW-NEO',
 'KRW-ONG',
 'KRW-ONT',
 'KRW-ORBS',
 'KRW-PLA',
 'KRW-POLYX',
 'KR